In [ ]:
import numpy as np
import cv2
import pandas as pd
import warnings
warnings.filterwarnings("ignore")

In [ ]:
class MyKNN:

  def __init__(self,K=3,weight_gist= 0.85/(0.85+0.75)):
    self.weight_gist = weight_gist
    self.K = K
    pass

  """fit: normalized后输入, 输入变量都化成list(np.ndarray)"""
  def fit(self,training_gist_set,training_hough_set,training_y):
    self.training_gist_set = training_gist_set
    self.training_hough_set = training_hough_set
    self.training_y = training_y
  
  def computeDistance(self,training_gist_v,training_hough_v,target_gist_vector,target_hough_vector):
    diff1 = np.sum((training_gist_v-target_gist_vector) ** 2) ** 0.5
    diff2 = np.sum((training_hough_v-target_hough_vector) ** 2) ** 0.5
    # diff2 = np.linalg.norm(training_hough_v-target_hough_vector,'L2')
    dis = self.weight_gist*diff1 + (1-self.weight_gist)*diff2
    return dis

  def getNeighbor(self,target_gist_vector,target_hough_vector):
    sample_dis_to_train = np.zeros(shape=len(self.training_gist_set))
    for i in range(0,len(self.training_gist_set)):
      sample_dis_to_train[i] = self.computeDistance(training_gist_v=self.training_gist_set[i],training_hough_v=self.training_hough_set[i],target_gist_vector=target_gist_vector,target_hough_vector=target_hough_vector)
    
    K_index = np.argsort(sample_dis_to_train,kind='stable')[:self.K]
    neighbors = self.training_y[K_index]
    return neighbors

  def majority(self, neighbors):
    b = np.bincount(neighbors)
    return np.argmax(b)

  def predict(self,target_gist_set,target_hough_set):
    target_predictions = []
    for i in range(0,len(target_gist_set)):
      ns = self.getNeighbor(target_gist_vector=target_gist_set[i],target_hough_vector=target_hough_set[i])
      res = self.majority(neighbors=ns)
      target_predictions.append(res)
    return target_predictions
  



In [ ]:
class GistUtils:
    def __init__(self, n_resize=128, n_w=5, ln_orientation=[8, 8, 8, 8], n_block_num=4, n_prefilt=4):
        # vector dim(single chanle) = sum(ln_orientation) * n_block_num * n_block_num
        # (8+8+8+8)*(4*4) = 512
        self.n_resize = n_resize
        self.n_boundaryExtension = self.n_resize // 4
        self.n_w = n_w    
        self.ln_orientation = ln_orientation
        self.n_block_num = n_block_num  # MUST n_resize % n_block_num == 0
        self.n_prefilt = n_prefilt
             
        self.__create_gabor()
        self.__get_gfmat()
        
    def get_gist_vec(self, np_img_raw, mode="rgb"):
        # resize 
        np_img_resize, n_ret = img_resize(np_img_raw, (self.n_resize, self.n_resize))
        if n_ret != 0:
            print("image resize error")
            return None

        # convert gray or rgb
        np_gist = None
        if mode.lower() == "gray":
            np_img_gray, n_ret = img_2gray(np_img_resize)
            np_prefilt_img = self.__get_pre_filt(np_img_gray)
            np_gist = self.__gist_main(np_prefilt_img)

        elif mode.lower() == "rgb" or mode.lower() == "bgr":
            np_img_bgr, n_ret = img_2bgr(np_img_resize)
            
            np_img_b = np_img_bgr[:,:,0]
            np_img_g = np_img_bgr[:,:,1]
            np_img_r = np_img_bgr[:,:,2]

            np_gist_b = self.__get_pre_filt(np_img_b)
            np_gist_g = self.__get_pre_filt(np_img_g)
            np_gist_r = self.__get_pre_filt(np_img_r)

            np_gist_b = self.__gist_main(np_gist_b)
            np_gist_g = self.__gist_main(np_gist_g)
            np_gist_r = self.__gist_main(np_gist_r)


            np_gist = np.hstack([np_gist_b, np_gist_g, np_gist_r])
        else:
            print("input mode error")
        
        return np_gist
    
    def __get_pre_filt(self, np_img): 
        np_log_img = np.log(np_img + 1.0)
        np_pad_img = np.pad(np_log_img,((self.n_w,self.n_w), (self.n_w,self.n_w)), 'symmetric')

        np_gf = self.np_gf
        np_out = np_pad_img - np.real(np.fft.ifft2(np.fft.fft2(np_pad_img) * np_gf ))
        
        np_local = np.sqrt(np.abs(np.fft.ifft2(np.fft.fft2(np_out **2) * np_gf)))
        np_out = np_out / (0.2 + np_local)
        
        n_size = self.n_resize + 2 * self.n_w
        
        return np_out[self.n_w: n_size - self.n_w, self.n_w : n_size - self.n_w]
    
    def __gist_main(self, np_prefilt_img):
        
        n_b = self.n_boundaryExtension
        np_pad_img = np.pad(np_prefilt_img, ((n_b, n_b), (n_b, n_b)), 'symmetric')
        np_fft2_img = np.fft.fft2(np_pad_img)
        
    
        n_filter = self.np_gabor.shape[2]      
        n_size = self.np_gabor.shape[0]
        lf_gist = []
        for i in range(n_filter):
            np_res = np.abs(np.fft.ifft2( np_fft2_img * self.np_gabor[:,:,i] ))
            
            np_res = np_res[n_b: n_size - n_b, n_b : n_size - n_b]
            
            lf_filter_res = self.__down_sampling(np_res)      
            lf_gist = lf_gist + lf_filter_res
        
        np_gist = np.asarray(lf_gist)
        return np_gist[np.newaxis,:]
    
    def __create_gabor(self):
        n_gabor_size = self.n_resize + 2 * self.n_boundaryExtension
        ln_or = self.ln_orientation
        
        n_scales = len(ln_or)
        n_filters = sum(ln_or)
        
        np_param = np.zeros((n_filters, 4), dtype = np.float64)
        n_index = 0
        for i in range(n_scales):
            for j in range(0, ln_or[i]):
                np_param[n_index, 0] = 0.35
                np_param[n_index, 1] = 0.3 / (1.85**i)
                np_param[n_index, 2] = 16 *(ln_or[i]**2)/(32**2)
                np_param[n_index, 3] = np.pi/ln_or[i] * j
                
                n_index += 1
         
     
        np_linear = np.linspace(-n_gabor_size//2, n_gabor_size//2-1, n_gabor_size)
        np_fx, np_fy = np.meshgrid(np_linear, np_linear)
        np_res_A = np.fft.fftshift(np.sqrt(np_fx ** 2 + np_fy**2))
        np_res_B = np.fft.fftshift(np.angle(np_fx + 1j*np_fy))
        
        self.np_gabor = np.zeros((n_gabor_size, n_gabor_size, n_filters), dtype = np.float64)
        for i in range(n_filters):
            np_tr = np_res_B + np_param[i,3]
            np_A  = (np_tr < -np.pi) + 0.0
            np_B  = (np_tr > np.pi) + 0.0
            
            np_tr = np_tr + 2 *np.pi * np_A - 2*np.pi*np_B
            np_every_gabor = np.exp(-10 * np_param[i,0] * ((np_res_A / n_gabor_size /np_param[i,1] - 1) **2) - 2*np_param[i,2]*np.pi*(np_tr **2))
            
            self.np_gabor[:,:,i] = np_every_gabor
            
            
    
    def __get_gfmat(self):
        n_s1 = self.n_prefilt /np.sqrt(np.log(2))
        n_boundray = self.n_resize + 2 * self.n_w
         
        np_linear = np.linspace(-n_boundray//2, n_boundray//2-1, n_boundray)
        np_fx, np_fy = np.meshgrid(np_linear, np_linear)
        
#        np_gf = np.fft.fftshift(np.exp( -(np_fx **2 + np_fy **2)/(n_s1 ** 2)))
        self.np_gf = np.fft.fftshift(np.exp( -(np_fx **2 + np_fy **2)/(n_s1 ** 2)))
           
    def __down_sampling(self, np_img):
        np_index = np.linspace(0, self.n_resize, self.n_block_num + 1, dtype = np.int)
        ln_data = []
        for i in range(self.n_block_num):
            for j in range(self.n_block_num):
                np_zone = np_img[np_index[i]: np_index[i+1] , np_index[j]: np_index[j+1]]
                np_zone = np_zone.T.reshape(-1)
#                n_res = np.median(np_zone)
                n_res = np.max(np_zone)
                ln_data.append(n_res)
        return ln_data



        
def img_2bgr(np_img_in, run_log=None, b_print=False):
    if np_img_in is None:
        return None, -3 

    np_img_bgr = None # if raw image is uint16 so conver to uint8 
    if len(np_img_in.shape) == 3 and np_img_in.shape[2] == 3: # Raw Image is BGR imge, so continue
        np_img_bgr = np_img_in
    elif len(np_img_in.shape) == 3 and np_img_in.shape[2] == 4: # Raw Image is BGRA imge, there are different situation to solve
        h, w, c = np_img_in.shape
        np_img_bgr_1 = cv2.cvtColor(np_img_in, cv2.COLOR_BGRA2BGR)    
        b, g, r, a = cv2.split(np_img_in)
        b = cv2.convertScaleAbs(b, alpha=(255.0/65535.0)) # (b/256).astype('uint8')
        g = cv2.convertScaleAbs(g, alpha=(255.0/65535.0))
        r = cv2.convertScaleAbs(r, alpha=(255.0/65535.0))
        a = cv2.convertScaleAbs(a, alpha=(255.0/65535.0))
        new_img  = cv2.merge((b, g, r))
        not_a = cv2.bitwise_not(a)
        not_a = cv2.cvtColor(not_a, cv2.COLOR_GRAY2BGR) 
        new_img = cv2.bitwise_and(new_img, new_img, mask = a)
        np_img_bgr_2 = cv2.add(new_img, not_a) 
        np_img_gray_1 = cv2.cvtColor(cv2.convertScaleAbs(np_img_bgr_1, alpha=(255.0/65535.0)), cv2.COLOR_BGR2GRAY) # Which image has most not white
        np_img_gray_2 = cv2.cvtColor(np_img_bgr_2, cv2.COLOR_BGR2GRAY)
        n_info_1 = len(np.unique(np_img_gray_1))
        n_info_2 = len(np.unique(np_img_gray_2))    
        np_img_bgr = np_img_bgr_1 if n_info_1 >= n_info_2 else np_img_bgr_2
    elif len(np_img_in.shape) == 3 and np_img_in.shape[2] == 1: # Raw Image is gray image
        np_img_bgr = np.tile(np_img_in, (1, 1, 3))     # 256x256x1 ==> 256x256x3
    elif len(np_img_in.shape) == 2:
        np_img_bgr = np.tile(np_img_in, (3, 1, 1))     # 256x256 ==> 3x256x256
        np_img_bgr = np.transpose(np_img_bgr, (1, 2, 0))  # 3x256x256 ==> 256x256x3
    return np_img_bgr, 0
    
''' 
Convert raw image to small gray image, resize is  n_resize * n_resize 
''' 
def img_2gray(np_img_raw, run_log=None, b_print=False):
    if np_img_raw is None:
        s_msg = "input image null"
        run_log and run_log.error(s_msg)
        b_print and print(s_msg)
        return None, -3
    np_img_gray = None # Raw Image is BGR imge, so convert rgb to gray
    if len(np_img_raw.shape) == 3 and np_img_raw.shape[2] == 3:
        np_img_gray = cv2.cvtColor(np_img_raw, cv2.COLOR_BGR2GRAY) 
    elif len(np_img_raw.shape) == 3 and np_img_raw.shape[2] == 4: # Raw Image is BGRA imge, there are different situation to solve
        n_sence = 3
        np_img_gray_choose = np.zeros([np_img_raw.shape[0], np_img_raw.shape[1], n_sence], dtype=np.uint8)

        np_img_gray_choose[:, :, 0] = 255 - np_img_raw[:, :, 3]
        np_img_gray_choose[:, :, 1] = cv2.cvtColor(np_img_raw, cv2.COLOR_BGRA2GRAY)
        np_img_gray_choose[:, :, 2] = cv2.cvtColor(np_img_raw[:, :, 0:3], cv2.COLOR_BGR2GRAY)
   
        ln_sence_non0_num = [] # Get nonzero element of every resize gray
        for i in range(n_sence):
            ln_sence_non0_num.append(len(np_img_gray_choose[:, :, i].nonzero()[0]))  
        if len(set(ln_sence_non0_num)) > 1: # Which image has most nonzero element
            n_max_index = ln_sence_non0_num.index(max(ln_sence_non0_num))
            np_img_gray = np_img_gray_choose[:, :, n_max_index]
        else: # Which image has most different element
            ln_diff_pix_num = []
            for i in range(n_sence):
                ln_diff_pix_num.append(len(np.unique(np_img_gray_choose[:, :, i])))
            n_max_index = ln_diff_pix_num.index(max(ln_diff_pix_num))
            np_img_gray = np_img_gray_choose[:, :, n_max_index] 
    elif len(np_img_raw.shape) == 3 and np_img_raw.shape[2] == 1: # Raw Image is gray image
        np_img_gray = np_img_raw[:, :, 0]
    elif len(np_img_raw.shape) == 2:
        np_img_gray = np_img_raw
    return np_img_gray, 0


''' 
resize a image 
'''     
def img_resize(np_img_in, ln_resize, run_log=None, b_print=False):
    try:
        np_img_resize = cv2.resize(np_img_in, ln_resize, fx=0.5, fy=0.5, interpolation=cv2.INTER_AREA)
        return np_img_resize, 0
    except Exception as e:
        s_msg = 'resize err:%s' % str(e)
        run_log and run_log.error(s_msg)
        b_print and print(s_msg)
        return None, -3

In [ ]:
class classificateForTarget():

  def __init__(self) -> None:
    self.gist_helper = GistUtils()
  
  def fit(self,X_train,y_train,K_list=[3,5,7],weight_gist = 84.2/(79.2+84.2)) -> None:
      self.X_train = X_train
      self.y_train = y_train
      self.K_list = K_list
      
      #pre-process
      X_train_gray_equ = []
      X_train_equ_blur =[]
      X_train_edge = []

      #灰度直方图均衡化
      for i in range(0,len(self.X_train)):
        img = cv2.resize(self.X_train[i],(256,256))
        img = self.X_train[i].astype(np.uint8)
        img = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
        X_train_gray_equ.append(cv2.equalizeHist(img))

      #高斯模糊
      for i in range(0,len(X_train_gray_equ)):
        X_train_equ_blur.append(cv2.GaussianBlur(X_train_gray_equ[i],(5,5),1.5))

      #canny
      for i in range(0,len(X_train_equ_blur)):
        X_train_edge.append(cv2.Canny(X_train_equ_blur[i],30,100))

      #prepare data for KNN train

      ## hough---
      X_train_hough = []
      X_train_hough_len = []
      self.th1 = 7
      self.maxg1 = 7.4
      self.th2 = 15
      self.maxg2 = 20

      for i in range(0,len(X_train_edge)):
        lines = cv2.HoughLinesP(X_train_edge[i], 1, np.pi/180, self.th1, 100, self.maxg1)
        if lines is None:
          X_train_hough.append(0)
        else:
          X_train_hough.append(len(lines))

        lines_for_length = cv2.HoughLinesP(X_train_edge[i], 1, np.pi/180, self.th2, 100, self.maxg2)
        max_length = 0 #曼哈顿距离
        if lines_for_length is not None:
          for line in lines_for_length:
            x1 = line[0][0]
            y1 = line[0][1]
            x2 = line[0][2]
            y2 = line[0][3]
            temp = y2-y1+x2-x1
            if temp > max_length:
              max_length = temp
          
        X_train_hough_len.append(max_length)

      X_train_hough_2 = pd.DataFrame(X_train_hough,columns=['number of lines'])
      X_train_hough_2["max length of line"] = X_train_hough_len

      self.minx_num = np.min(X_train_hough_2['number of lines'])
      self.minx_len = np.min(X_train_hough_2['max length of line'])
      self.maxx_num = np.max(X_train_hough_2['number of lines'])
      self.maxx_len = np.max(X_train_hough_2['max length of line'])
      
      X_train_hough_2_ = X_train_hough_2.apply(lambda x:(x-np.min(x))/(np.max(x)-np.min(x)))

      self.X_train_hough_set = []
      for i in range(0,len(X_train_hough)):
        self.X_train_hough_set.append(np.array([X_train_hough_2_.iat[i,0],X_train_hough_2_.iat[i,1]]))

      ## gist---
      self.X_train_gist_ = []

      for i in range(0,len(self.X_train)):
        self.X_train_gist_.append(self.gist_helper.get_gist_vec(self.X_train[i]))

      # # knn fit
      # self.knn = MyKNN(weight_gist = 84.2/(79.2+84.2))
      # self.knn.fit(self.X_train_gist_,self.X_train_hough_set,np.array(y_train))

      self.knn_list = []
      for i in range(0, len(K_list)):
        knn = MyKNN(K=K_list[i],weight_gist = weight_gist)
        knn.fit(self.X_train_gist_, self.X_train_hough_set,np.array(self.y_train))
        self.knn_list.append(knn)

  def emsemlble_knn(self):
    pred = []
    for i in range(0,len(self.K_list)):
      pred_myknn = self.knn_list[i].predict(self.gist,self.hough_set)
      pred.append(np.array(pred_myknn))

    sum_pred = 0
    for l in range(0,len(pred)):
      sum_pred = sum_pred + pred[i]

    predict = np.round(sum_pred/len(self.K_list))

    return predict

  def preprocess_target(self, target_img) -> None:
      target_img1 = cv2.resize(target_img,(256,256))
      target_img1 = cv2.cvtColor(target_img1, cv2.COLOR_RGB2GRAY)
      target_img1 = cv2.equalizeHist(target_img1)
      
      blur = cv2.GaussianBlur(target_img1,(5,5),1.5)
      edge = cv2.Canny(blur,30,100)

      self.target_hough_len = 0
      self.target_hough_num = 0
      
      lines = cv2.HoughLinesP(edge, 1, np.pi/180, self.th1, 100, self.maxg1)
      if lines is not None:
        self.target_hough_num = len(lines)
      else:
        self.target_hough_num = 0
      self.target_hough_num = (self.target_hough_num - self.minx_num)/(self.maxx_num - self.maxx_num)

      lines_for_length = cv2.HoughLinesP(edge, 1, np.pi/180, self.th2, 100, self.maxg2)
      max_length = 0 
      if lines_for_length is not None:
        for line in lines_for_length:
          x1 = line[0][0]
          y1 = line[0][1]
          x2 = line[0][2]
          y2 = line[0][3]
          temp = y2-y1+x2-x1
          if temp > max_length:
            max_length = temp
        
      self.target_hough_len = max_length
      self.target_hough_len = (self.target_hough_len - self.minx_len)/(self.maxx_len - self.maxx_len)
      
      #final for input of KNN
      self.hough_set = [np.array([self.target_hough_num,self.target_hough_len])]
      self.gist = self.gist_helper.get_gist_vec(target_img)

  def predict(self, target_img) -> int:
    self.preprocess_target(target_img)
    pred = self.emsemlble_knn()
    return pred


In [ ]:
# c = classificateForTarget()
# c.fit(X_train,y_train)


In [ ]:
# p = c.predict(X_test[8])
# p